<h1>Swiss Federal Office of statistics</h1>
This project uses the data that is provided from Federal Statistic Office of buildings and Dwellings <a href=https://www.housing-stat.ch/fr/madd/public.html>can be found here</a>. It maps the metadata with the help of <a href=https://www.i14y.admin.ch/fr/catalog/all?types=DataService>I14Y</a> and creates the relationship in a Neo4J Cloud Database (AuraDB).

<h3>Getting the metadata</h3>
We would take a sample response for a building with EGID = 20, in order to get the metadata that are also explained in the E-Goverment-Standards 
<a href=https://www.ech.ch/sites/default/files/dosvers/hauptdokument/STAN_d_DEF_2022-06-18_eCH-0206_V2.0.0_GWR-Daten_an_Dritte.pdf> pdf </a> .

In [38]:
!pip install requests

import requests

# Define the URL for the web service
url = 'https://madd.bfs.admin.ch/eCH-0206'

# Define the XML payload
xml_payload = '''<?xml version="1.0" encoding="UTF-8"?>
<eCH-0206:maddRequest xmlns:eCH-0058="http://www.ech.ch/xmlns/eCH-0058/5" xmlns:eCH-0206="http://www.ech.ch/xmlns/eCH-0206/2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.ech.ch/xmlns/eCH-0206/2 eCH-0206-2-0-draft.xsd">
   <eCH-0206:requestHeader>
      <eCH-0206:messageId>myMessageId</eCH-0206:messageId>
      <eCH-0206:businessReferenceId>BFS/OFS/UST</eCH-0206:businessReferenceId>
      <eCH-0206:requestingApplication>
         <eCH-0058:manufacturer>yourAppName</eCH-0058:manufacturer>
         <eCH-0058:product>yourProductName</eCH-0058:product>
         <eCH-0058:productVersion>1.0</eCH-0058:productVersion>
      </eCH-0206:requestingApplication>
      <eCH-0206:comment>EGID=20, Dataset 3</eCH-0206:comment>
      <eCH-0206:requestDate>2024-07-27T00:00:00Z</eCH-0206:requestDate>
  </eCH-0206:requestHeader>
  <eCH-0206:requestContext>building</eCH-0206:requestContext>
  <eCH-0206:requestQuery>
    <eCH-0206:EGID>20</eCH-0206:EGID>
  </eCH-0206:requestQuery>
</eCH-0206:maddRequest>'''

# Set the headers
headers = {
    'Content-Type': 'text/xml'
}

# Send the POST request
response = requests.post(url, data=xml_payload, headers=headers)

# Check the response status
if response.status_code == 200:
    # Print the response XML or handle it as needed
    print(response.text)
else:
    print(f'Error: {response.status_code}')
    print(response.text)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


<?xml version="1.0" encoding="utf-8"?>
<maddResponse xmlns="http://www.ech.ch/xmlns/eCH-0206/2" xmlns:eCH-0058="http://www.ech.ch/xmlns/eCH-0058/5" xmlns:eCH-0129="http://www.ech.ch/xmlns/eCH-0129/5" xmlns:eCH-0007="http://www.ech.ch/xmlns/eCH-0007/6"><status><code>100</code><message>OK</message></status><responseHeader><messageId>80e195f0-535a-11ef-837c-0242ac140006</messageId><requestMessageId>myMessageId</requestMessageId><businessReferenceId>BFS/OFS/UST</businessReferenceId><respondingApplication><eCH-0058:manufacturer>SFSO</eCH-0058:manufacturer><eCH-0058:product>MADD</eCH-0058:product><eCH-0058:productVersion>0.1</eCH-0058:productVersion></respondingApplication><comment>no comment</comment><responseDate>2024-08-05T18:42:44</responseDate></responseHeader><originalRequest xmlns:eCH-0058="http://www.ech.ch/xmlns/eCH-0058/5" xmlns:eCH-0206="http://www.ech.ch/xmlns/eCH-0206/2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.ech.ch/xmlns/eCH-0206/2 

In [2]:
import xml.etree.ElementTree as ET
import pprint

# Parse the XML text
root = ET.fromstring(response.text)

# Namespace dictionary to help with finding elements with namespace
namespaces = {
    'ns': 'http://www.ech.ch/xmlns/eCH-0206/2',
    'eCH-0058': 'http://www.ech.ch/xmlns/eCH-0058/5',
    'eCH-0129': 'http://www.ech.ch/xmlns/eCH-0129/5',
    'eCH-0007': 'http://www.ech.ch/xmlns/eCH-0007/6'
}

# Extract the relevant portion (building-related information)
building_list = root.find('ns:buildingList', namespaces)

# Convert the building-related information to a dictionary
def xml_to_dict(element):
    return {child.tag.split('}', 1)[1]: xml_to_dict(child) if list(child) else child.text for child in element}

building_dict = xml_to_dict(building_list)


pprint.pprint(building_dict)

{'buildingItem': {'EGID': '20',
                  'building': {'buildingCategory': '1020',
                               'buildingClass': '1110',
                               'buildingStatus': '1004',
                               'coordinates': {'east': '2676418.734',
                                               'north': '1236038.458',
                                               'originOfCoordinates': '901'},
                               'dateOfConstruction': {'dateOfConstruction': '1981',
                                                      'periodOfConstruction': '8016'},
                               'numberOfFloors': '2',
                               'officialBuildingNo': '1145',
                               'recordModification': {'createDate': '2001-11-29',
                                                      'updateDate': '2021-03-15'},
                               'surfaceAreaOfBuilding': '100',
                               'thermotechnicalDeviceForHeating

We extract the keys from the api call that returned us a json object. This is an easy way to create a data dictionary, that will map the metadata found from one site with the metadata that the extraction of the SQL Lite Data RegBl provides.

In [39]:
def extract_keys(d, keys_list=None):
    if keys_list is None:
        keys_list = []
    if isinstance(d, dict):
        for key, value in d.items():
            keys_list.append(key)
            extract_keys(value, keys_list)
    return keys_list

# Get all keys
keys = extract_keys(building_dict)
print(keys)

['buildingItem', 'EGID', 'building', 'officialBuildingNo', 'coordinates', 'east', 'north', 'originOfCoordinates', 'buildingStatus', 'buildingCategory', 'buildingClass', 'dateOfConstruction', 'dateOfConstruction', 'periodOfConstruction', 'surfaceAreaOfBuilding', 'numberOfFloors', 'thermotechnicalDeviceForHeating1', 'heatGeneratorHeating', 'energySourceHeating', 'informationSourceHeating', 'revisionDate', 'thermotechnicalDeviceForWarmWater1', 'heatGeneratorHotWater', 'energySourceHeating', 'informationSourceHeating', 'revisionDate', 'thermotechnicalDeviceForWarmWater2', 'heatGeneratorHotWater', 'energySourceHeating', 'informationSourceHeating', 'revisionDate', 'recordModification', 'createDate', 'updateDate', 'buildingEntranceList', 'buildingEntranceItem', 'EDID', 'buildingEntrance', 'EGAID', 'buildingEntranceNo', 'coordinates', 'east', 'north', 'isOfficialAddress', 'recordModification', 'createDate', 'updateDate', 'street', 'ESID', 'isOfficialDescription', 'streetNameList', 'streetNameI

In [40]:
del keys[0] 

In [41]:
keys

['EGID',
 'building',
 'officialBuildingNo',
 'coordinates',
 'east',
 'north',
 'originOfCoordinates',
 'buildingStatus',
 'buildingCategory',
 'buildingClass',
 'dateOfConstruction',
 'dateOfConstruction',
 'periodOfConstruction',
 'surfaceAreaOfBuilding',
 'numberOfFloors',
 'thermotechnicalDeviceForHeating1',
 'heatGeneratorHeating',
 'energySourceHeating',
 'informationSourceHeating',
 'revisionDate',
 'thermotechnicalDeviceForWarmWater1',
 'heatGeneratorHotWater',
 'energySourceHeating',
 'informationSourceHeating',
 'revisionDate',
 'thermotechnicalDeviceForWarmWater2',
 'heatGeneratorHotWater',
 'energySourceHeating',
 'informationSourceHeating',
 'revisionDate',
 'recordModification',
 'createDate',
 'updateDate',
 'buildingEntranceList',
 'buildingEntranceItem',
 'EDID',
 'buildingEntrance',
 'EGAID',
 'buildingEntranceNo',
 'coordinates',
 'east',
 'north',
 'isOfficialAddress',
 'recordModification',
 'createDate',
 'updateDate',
 'street',
 'ESID',
 'isOfficialDescription'

In [42]:
data_dictionary = {key: None for key in keys}
data_dictionary

{'EGID': None,
 'building': None,
 'officialBuildingNo': None,
 'coordinates': None,
 'east': None,
 'north': None,
 'originOfCoordinates': None,
 'buildingStatus': None,
 'buildingCategory': None,
 'buildingClass': None,
 'dateOfConstruction': None,
 'periodOfConstruction': None,
 'surfaceAreaOfBuilding': None,
 'numberOfFloors': None,
 'thermotechnicalDeviceForHeating1': None,
 'heatGeneratorHeating': None,
 'energySourceHeating': None,
 'informationSourceHeating': None,
 'revisionDate': None,
 'thermotechnicalDeviceForWarmWater1': None,
 'heatGeneratorHotWater': None,
 'thermotechnicalDeviceForWarmWater2': None,
 'recordModification': None,
 'createDate': None,
 'updateDate': None,
 'buildingEntranceList': None,
 'buildingEntranceItem': None,
 'EDID': None,
 'buildingEntrance': None,
 'EGAID': None,
 'buildingEntranceNo': None,
 'isOfficialAddress': None,
 'street': None,
 'ESID': None,
 'isOfficialDescription': None,
 'streetNameList': None,
 'streetNameItem': None,
 'language': No

<h2>Creating the data dictionary</h2>
That is how we will map the metadata we get from the request that we did above, with the name of the columns that are provided from the RegBl.

In [43]:
data_dictionary['EGID'] = 'EGID'
data_dictionary['officialBuildingNo'] = 'GEBNR'
data_dictionary['east'] = 'GKODE'
data_dictionary['north'] = 'GKODN'
data_dictionary['originOfCoordinates'] = 'GKSCE'
data_dictionary['buildingStatus'] = 'GSTAT'
data_dictionary['buildingCategory'] = 'GKAT'
data_dictionary['buildingClass'] = 'GKLAS'
data_dictionary['dateOfConstruction'] = 'GBAUJM'
data_dictionary['periodOfConstruction'] = 'GBAUP'
data_dictionary['surfaceAreaOfBuilding'] = 'GAREA'
data_dictionary['numberOfFloors'] = 'GASTW'
data_dictionary['heatGeneratorHeating'] = 'GASTW'
data_dictionary['energySourceHeating'] = 'GENW1'
data_dictionary['informationSourceHeating'] = 'GWAERSCEW1'
data_dictionary['revisionDate'] = 'GWAERDATH1'
data_dictionary['heatGeneratorHotWater'] = 'GWAERZW1'
data_dictionary['createDate'] = 'Create_Date'
data_dictionary['updateDate'] = 'Update_Date'
data_dictionary['EDID'] = 'EDID'
data_dictionary['EGAID'] = 'EGAID'
data_dictionary['buildingEntranceNo'] = 'DEINR'
data_dictionary['isOfficialAddress'] = 'DOFFADR'
data_dictionary['ESID'] = 'ESID'
data_dictionary['EGAID'] = 'EGAID'
data_dictionary['buildingEntranceNo'] = 'DEINR'
data_dictionary['isOfficialDescription'] = 'STROFFIZIEL'
data_dictionary['language'] = 'STRSP'
data_dictionary['descriptionLong'] = 'STRNAME'
data_dictionary['descriptionShort'] = 'STRNAMK'
data_dictionary['swissZipCode'] = 'DPLZ4'
data_dictionary['swissZipCodeAddOn'] = 'DPLZZ'
data_dictionary['placeName'] = 'DPLZNAME'
data_dictionary['EWID'] = 'EWID'
data_dictionary['yearOfConstruction'] = 'WBAUJ'
data_dictionary['noOfHabitableRooms'] = 'WAZIM'
data_dictionary['floor'] = 'WSTWK'
data_dictionary['multipleFloor'] = 'WMEHRG'
data_dictionary['kitchen'] = 'WKCHE'
data_dictionary['surfaceAreaOfDwelling'] = 'WAREA'
data_dictionary['dwellingStatus'] = 'WSTAT'
data_dictionary['municipalityId'] = 'GGDENR'
data_dictionary['cantonAbbreviation'] = 'GDEKT'
data_dictionary['EGRID'] = 'EGRID'
data_dictionary['number'] = 'LPARZ'
data_dictionary['subDistrict'] = 'LGBKR'

In [44]:
data_dictionary

{'EGID': 'EGID',
 'building': None,
 'officialBuildingNo': 'GEBNR',
 'coordinates': None,
 'east': 'GKODE',
 'north': 'GKODN',
 'originOfCoordinates': 'GKSCE',
 'buildingStatus': 'GSTAT',
 'buildingCategory': 'GKAT',
 'buildingClass': 'GKLAS',
 'dateOfConstruction': 'GBAUJM',
 'periodOfConstruction': 'GBAUP',
 'surfaceAreaOfBuilding': 'GAREA',
 'numberOfFloors': 'GASTW',
 'thermotechnicalDeviceForHeating1': None,
 'heatGeneratorHeating': 'GASTW',
 'energySourceHeating': 'GENW1',
 'informationSourceHeating': 'GWAERSCEW1',
 'revisionDate': 'GWAERDATH1',
 'thermotechnicalDeviceForWarmWater1': None,
 'heatGeneratorHotWater': 'GWAERZW1',
 'thermotechnicalDeviceForWarmWater2': None,
 'recordModification': None,
 'createDate': 'Create_Date',
 'updateDate': 'Update_Date',
 'buildingEntranceList': None,
 'buildingEntranceItem': None,
 'EDID': 'EDID',
 'buildingEntrance': None,
 'EGAID': 'EGAID',
 'buildingEntranceNo': 'DEINR',
 'isOfficialAddress': 'DOFFADR',
 'street': None,
 'ESID': 'ESID',
 

In [45]:
swapped_data_dict = {value: key for key, value in data_dictionary.items()}

In [46]:
swapped_data_dict

{'EGID': 'EGID',
 None: 'realestateIdentificationItem',
 'GEBNR': 'officialBuildingNo',
 'GKODE': 'east',
 'GKODN': 'north',
 'GKSCE': 'originOfCoordinates',
 'GSTAT': 'buildingStatus',
 'GKAT': 'buildingCategory',
 'GKLAS': 'buildingClass',
 'GBAUJM': 'dateOfConstruction',
 'GBAUP': 'periodOfConstruction',
 'GAREA': 'surfaceAreaOfBuilding',
 'GASTW': 'heatGeneratorHeating',
 'GENW1': 'energySourceHeating',
 'GWAERSCEW1': 'informationSourceHeating',
 'GWAERDATH1': 'revisionDate',
 'GWAERZW1': 'heatGeneratorHotWater',
 'Create_Date': 'createDate',
 'Update_Date': 'updateDate',
 'EDID': 'EDID',
 'EGAID': 'EGAID',
 'DEINR': 'buildingEntranceNo',
 'DOFFADR': 'isOfficialAddress',
 'ESID': 'ESID',
 'STROFFIZIEL': 'isOfficialDescription',
 'STRSP': 'language',
 'STRNAME': 'descriptionLong',
 'STRNAMK': 'descriptionShort',
 'DPLZ4': 'swissZipCode',
 'DPLZZ': 'swissZipCodeAddOn',
 'DPLZNAME': 'placeName',
 'EWID': 'EWID',
 'WBAUJ': 'yearOfConstruction',
 'WAZIM': 'noOfHabitableRooms',
 'WSTWK':

<h2>Building has Entrance Relationship</h2>
To start with, we will show this relationship for only the canton of Geneva. This means that we have downloaded the files that are needed from RegBl.

In [47]:
import pandas as pd
entrances_df = pd.read_csv('C:\\Users\\User\\Desktop\\KOs\\geneva_canton\\eingang_entree_entrata.csv',sep='\t')

entrances_geneva = entrances_df.head(30)

In [48]:
# Step 1: Keep only columns that are in the dictionary
columns_to_keep = [col for col in entrances_geneva.columns if col in swapped_data_dict]

# Step 2: Filter the DataFrame to keep only the desired columns
entrances_geneva = entrances_geneva[columns_to_keep]

# Step 3: Rename the columns using the dictionary
entrances_geneva.rename(columns=swapped_data_dict, inplace=True)

In [49]:
entrances_geneva

,EGID,EDID,EGAID,buildingEntranceNo,ESID,descriptionLong,descriptionShort,language,isOfficialDescription,swissZipCode,swissZipCodeAddOn,placeName,isOfficialAddress
0,203840,0,100206369,16,10178040.0,Boulevard de Saint-Georges,Bd de Saint-Georges,9903.0,1.0,1205,0,Genève,0.0
1,1000001,0,100678822,2,10095712.0,Chemin des Avallons,Ch. des Avallons,9903.0,1.0,1247,0,Anières,1.0
2,1000002,0,100678823,4,10095712.0,Chemin des Avallons,Ch. des Avallons,9903.0,1.0,1247,0,Anières,1.0
3,1000003,0,100678824,7,10095712.0,Chemin des Avallons,Ch. des Avallons,9903.0,1.0,1247,0,Anières,0.0
4,1000004,0,100678825,11,10095712.0,Chemin des Avallons,Ch. des Avallons,9903.0,1.0,1247,0,Anières,0.0
5,1000005,0,100678826,NaN,10095712.0,Chemin des Avallons,Ch. des Avallons,9903.0,1.0,1247,0,Anières,0.0
6,1000006,0,100678827,15,10095712.0,Chemin des Avallons,Ch. des Avallons,9903.0,1.0,1247,0,Anières,0.0
7,1000007,0,100678828,17,10095712.0,Chemin des Avallons,Ch. des Avallons,9903.0,1.0,1247,0,Anières,1.0
8,1000008,0,100678829,23,10095712.0,Chemin des Avallons,Ch. des Avallons,9903.0,1.0,1247,0,Anières,1.0
9,1000009,0,100678830,27,10095712.0,Chemin des Avallons,Ch. des Avallons,9903.0,1.0,1247,0,Anières,1.0


The data of the entrances that we have are a few, in this case we are trying to take only the buildings that the aforementioned entrances belonged to, in order to visualize relationships with meaning.

In [51]:
geneva_build = pd.read_csv('C:\\Users\\User\\Desktop\\KOs\\geneva_canton\\gebaeude_batiment_edificio.csv',sep='\t')

id_list = entrances_geneva['EGID'].tolist()

# Step 2: Construct the dynamic where clause
# Note: To prevent SQL injection, ensure that the ids are safe and clean.
query_str = "EGID in @id_list"

# Step 3: Apply the query to filter the DataFrame
geneva_build = geneva_build.query(query_str)

This is a function that can be used everytime we would like to translate the columns from the RegBl website to the e-goverment standarts.

In [52]:
def metadata_translation(df, columns_dict):
    """
    Filters and renames columns in a DataFrame based on a given dictionary.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    columns_dict (dict): A dictionary where the keys are the original column names 
                         to keep and the values are the new column names.
                         
    Returns:
    pd.DataFrame: The DataFrame with filtered and renamed columns.
    """
    # Step 1: Filter the DataFrame to keep only the desired columns
    columns_to_keep = [col for col in df.columns if col in columns_dict]
    
    df_filtered = df[columns_to_keep].copy()  # Explicitly make a copy to avoid SettingWithCopyWarning
    
    # Step 3: Rename the columns using the dictionary
    df_filtered.rename(columns=columns_dict, inplace=True)
    
    return df_filtered

geneva_buildings = metadata_translation(geneva_build,swapped_data_dict)
geneva_buildings

,EGID,cantonAbbreviation,municipalityId,EGRID,subDistrict,number,officialBuildingNo,east,north,originOfCoordinates,buildingStatus,buildingCategory,buildingClass,periodOfConstruction,surfaceAreaOfBuilding,heatGeneratorHeating,revisionDate,heatGeneratorHotWater,energySourceHeating,informationSourceHeating
451,1000001,GE,6602,CH936581631957,2.0,5068.0,1682,2506071.38,1125077.00,905,1004,1020.0,1110.0,8017.0,108.0,2.0,2001-11-29,7630.0,7530.0,860.0
452,1000002,GE,6602,CH883065638926,2.0,4855.0,862,2506093.63,1125043.50,905,1004,1020.0,1110.0,8014.0,79.0,2.0,2016-06-10,7630.0,7530.0,855.0
453,1000003,GE,6602,CH728836656337,2.0,4794.0,799,2506105.25,1125104.88,909,1007,1020.0,1110.0,8014.0,96.0,1.0,2001-11-29,7650.0,7560.0,860.0
454,1000004,GE,6602,CH336592634143,2.0,6290.0,1011,2506150.39,1125093.76,909,1007,1020.0,1110.0,8015.0,127.0,2.0,2001-11-29,7630.0,7530.0,860.0
455,1000005,GE,6602,CH528676656390,2.0,6264.0,824,2506165.01,1125141.97,905,1007,1020.0,1110.0,8014.0,118.0,1.0,2001-11-29,7630.0,7530.0,869.0
456,1000006,GE,6602,CH208663846524,2.0,6283.0,726,2506177.40,1125184.36,909,1007,1020.0,1110.0,8013.0,195.0,1.0,2001-11-29,7650.0,7560.0,860.0
457,1000007,GE,6602,CH116330658955,2.0,5311.0,1235,2506190.50,1125231.88,905,1004,1020.0,1110.0,8016.0,126.0,2.0,2001-11-29,7650.0,7560.0,860.0
458,1000008,GE,6602,CH516376658690,2.0,6263.0,1060,2506216.50,1125344.25,905,1004,1020.0,1110.0,8015.0,146.0,2.0,2001-11-29,7630.0,7530.0,860.0
459,1000009,GE,6602,CH328087656327,2.0,4931.0,757,2506243.88,1125414.63,905,1004,1020.0,1110.0,8013.0,89.0,3.0,2016-06-10,7650.0,7560.0,860.0
460,1000010,GE,6602,CH557565846389,2.0,6174.0,879,2506258.38,1125454.13,909,1007,1020.0,1110.0,8014.0,168.0,1.0,2001-11-29,7630.0,7530.0,860.0


In [53]:
geneva_buildings = geneva_buildings.fillna(0)
geneva_buildings

,EGID,cantonAbbreviation,municipalityId,EGRID,subDistrict,number,officialBuildingNo,east,north,originOfCoordinates,buildingStatus,buildingCategory,buildingClass,periodOfConstruction,surfaceAreaOfBuilding,heatGeneratorHeating,revisionDate,heatGeneratorHotWater,energySourceHeating,informationSourceHeating
451,1000001,GE,6602,CH936581631957,2.0,5068.0,1682,2506071.38,1125077.00,905,1004,1020.0,1110.0,8017.0,108.0,2.0,2001-11-29,7630.0,7530.0,860.0
452,1000002,GE,6602,CH883065638926,2.0,4855.0,862,2506093.63,1125043.50,905,1004,1020.0,1110.0,8014.0,79.0,2.0,2016-06-10,7630.0,7530.0,855.0
453,1000003,GE,6602,CH728836656337,2.0,4794.0,799,2506105.25,1125104.88,909,1007,1020.0,1110.0,8014.0,96.0,1.0,2001-11-29,7650.0,7560.0,860.0
454,1000004,GE,6602,CH336592634143,2.0,6290.0,1011,2506150.39,1125093.76,909,1007,1020.0,1110.0,8015.0,127.0,2.0,2001-11-29,7630.0,7530.0,860.0
455,1000005,GE,6602,CH528676656390,2.0,6264.0,824,2506165.01,1125141.97,905,1007,1020.0,1110.0,8014.0,118.0,1.0,2001-11-29,7630.0,7530.0,869.0
456,1000006,GE,6602,CH208663846524,2.0,6283.0,726,2506177.40,1125184.36,909,1007,1020.0,1110.0,8013.0,195.0,1.0,2001-11-29,7650.0,7560.0,860.0
457,1000007,GE,6602,CH116330658955,2.0,5311.0,1235,2506190.50,1125231.88,905,1004,1020.0,1110.0,8016.0,126.0,2.0,2001-11-29,7650.0,7560.0,860.0
458,1000008,GE,6602,CH516376658690,2.0,6263.0,1060,2506216.50,1125344.25,905,1004,1020.0,1110.0,8015.0,146.0,2.0,2001-11-29,7630.0,7530.0,860.0
459,1000009,GE,6602,CH328087656327,2.0,4931.0,757,2506243.88,1125414.63,905,1004,1020.0,1110.0,8013.0,89.0,3.0,2016-06-10,7650.0,7560.0,860.0
460,1000010,GE,6602,CH557565846389,2.0,6174.0,879,2506258.38,1125454.13,909,1007,1020.0,1110.0,8014.0,168.0,1.0,2001-11-29,7630.0,7530.0,860.0


In [54]:
import rdflib
from neo4j import GraphDatabase

uri = "neo4j+s://dc88e027.databases.neo4j.io:7687"  # Update with your Neo4j URI
user = "neo4j"  # Update with your Neo4j username
password = "fvzNWNsrsSdgsVTLL0dEEN3_E6zmxadKL38zx1GSxkY"  # Update with your Neo4j password
driver = GraphDatabase.driver(uri, auth=(user, password))

def create_building(tx, **properties):
    query = "MERGE (b:building {" + ", ".join(f"{key}: ${key}" for key in properties.keys()) + "})"
    tx.run(query, **properties)

def add_buildings_to_graph(df):
    with driver.session() as session:
        for index, row in df.iterrows():
            properties = {key: row[key] for key in df.columns}
            session.write_transaction(create_building, **properties)

add_buildings_to_graph(geneva_buildings)

C:\Users\User\AppData\Local\Temp\ipykernel_25024\666272353.py:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_building, **properties)


We wll create now functions to add nodes in general so we do not have to write every time, a new function for a different type of node.

In [55]:
def create_node(tx, label, **properties):
    # Construct the Cypher query using f-string correctly
    query = "MERGE (b:" + label + "{" + ", ".join(f"{key}: ${key}" for key in properties.keys()) + "})"
    tx.run(query, **properties)

def add_nodes_to_graph(df,label):
    df = df.fillna(0)
    with driver.session() as session:
        for index, row in df.iterrows():
            properties = {key: row[key] for key in df.columns}
            session.write_transaction(create_node,label, **properties)

add_nodes_to_graph(entrances_geneva, 'entrance')

C:\Users\User\AppData\Local\Temp\ipykernel_25024\3893174686.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node,label, **properties)


We will also create generic fuction for importing the relationship to the graph database. So to build any relationship in the future we will only need to parse the cypher query.

In [56]:
query = """
        MATCH (b:building), (e:entrance)
        WHERE b.EGID = e.EGID
        MERGE (b)-[:HasEntrance]->(e)
    """
def create_relationship(tx,query):
    tx.run(query)

def add_relationships_to_graph(query):
    with driver.session() as session:
        session.write_transaction(create_relationship,query)

add_relationships_to_graph(query)

C:\Users\User\AppData\Local\Temp\ipykernel_25024\3939598042.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship,query)


<h2>Building has Dwelling Relationship</h2>
Visualizing this type of relationship will also help us to prove that our concept is easily reusable in the future. When we got the response from the API there was no EWID included, so our dictionary of metadata does not have it inside to translateit. As a result we will just add that one key that we need to our already existig dictionary.

In [57]:
swapped_data_dict['EWID'] = 'EWID'
swapped_data_dict #checking that was added

{'EGID': 'EGID',
 None: 'realestateIdentificationItem',
 'GEBNR': 'officialBuildingNo',
 'GKODE': 'east',
 'GKODN': 'north',
 'GKSCE': 'originOfCoordinates',
 'GSTAT': 'buildingStatus',
 'GKAT': 'buildingCategory',
 'GKLAS': 'buildingClass',
 'GBAUJM': 'dateOfConstruction',
 'GBAUP': 'periodOfConstruction',
 'GAREA': 'surfaceAreaOfBuilding',
 'GASTW': 'heatGeneratorHeating',
 'GENW1': 'energySourceHeating',
 'GWAERSCEW1': 'informationSourceHeating',
 'GWAERDATH1': 'revisionDate',
 'GWAERZW1': 'heatGeneratorHotWater',
 'Create_Date': 'createDate',
 'Update_Date': 'updateDate',
 'EDID': 'EDID',
 'EGAID': 'EGAID',
 'DEINR': 'buildingEntranceNo',
 'DOFFADR': 'isOfficialAddress',
 'ESID': 'ESID',
 'STROFFIZIEL': 'isOfficialDescription',
 'STRSP': 'language',
 'STRNAME': 'descriptionLong',
 'STRNAMK': 'descriptionShort',
 'DPLZ4': 'swissZipCode',
 'DPLZZ': 'swissZipCodeAddOn',
 'DPLZNAME': 'placeName',
 'EWID': 'EWID',
 'WBAUJ': 'yearOfConstruction',
 'WAZIM': 'noOfHabitableRooms',
 'WSTWK':

In [58]:
geneva_dwel = pd.read_csv('C:\\Users\\User\\Desktop\\KOs\\geneva_canton\\wohnung_logement_abitazione.csv',sep='\t')
geneva_dwel.head(10)

,EGID,EWID,EDID,WHGNR,WEINR,WSTWK,WBEZ,WMEHRG,WBAUJ,WABBJ,WSTAT,WAREA,WAZIM,WKCHE,WEXPDAT
0,203840,1,0,NaN,NaN,3100.0,NaN,1.0,NaN,2009.0,3007,160.0,6.0,1.0,2024-06-05
1,1000001,2,0,E00.1,NaN,3100.0,NaN,1.0,1999.0,NaN,3004,160.0,6.0,1.0,2024-06-05
2,1000002,1,0,E00.1,NaN,3100.0,NaN,1.0,1999.0,NaN,3004,60.0,4.0,1.0,2024-06-05
3,1000003,1,0,E00.1,NaN,3100.0,NaN,0.0,1999.0,2012.0,3007,96.0,4.0,1.0,2024-06-05
4,1000004,1,0,E00.1,NaN,3100.0,NaN,1.0,1999.0,2022.0,3007,277.0,9.0,1.0,2024-06-05
5,1000005,1,0,E00.1,NaN,3100.0,NaN,0.0,1999.0,2023.0,3007,90.0,3.0,1.0,2024-06-05
6,1000006,1,0,E00.1,NaN,3100.0,NaN,0.0,1999.0,2022.0,3007,190.0,4.0,1.0,2024-06-05
7,1000007,1,0,E00.1,NaN,3100.0,NaN,1.0,1999.0,NaN,3004,200.0,5.0,1.0,2024-06-05
8,1000008,1,0,E00.1,NaN,3100.0,NaN,1.0,1999.0,NaN,3004,300.0,8.0,1.0,2024-06-05
9,1000009,1,0,E00.1,NaN,3100.0,NaN,1.0,1999.0,NaN,3004,196.0,7.0,1.0,2024-06-05


In [59]:
geneva_dwel = geneva_dwel.query(query_str)
geneva_dwel

,EGID,EWID,EDID,WHGNR,WEINR,WSTWK,WBEZ,WMEHRG,WBAUJ,WABBJ,WSTAT,WAREA,WAZIM,WKCHE,WEXPDAT
0,203840,1,0,NaN,NaN,3100.0,NaN,1.0,NaN,2009.0,3007,160.0,6.0,1.0,2024-06-05
1,1000001,2,0,E00.1,NaN,3100.0,NaN,1.0,1999.0,NaN,3004,160.0,6.0,1.0,2024-06-05
2,1000002,1,0,E00.1,NaN,3100.0,NaN,1.0,1999.0,NaN,3004,60.0,4.0,1.0,2024-06-05
3,1000003,1,0,E00.1,NaN,3100.0,NaN,0.0,1999.0,2012.0,3007,96.0,4.0,1.0,2024-06-05
4,1000004,1,0,E00.1,NaN,3100.0,NaN,1.0,1999.0,2022.0,3007,277.0,9.0,1.0,2024-06-05
5,1000005,1,0,E00.1,NaN,3100.0,NaN,0.0,1999.0,2023.0,3007,90.0,3.0,1.0,2024-06-05
6,1000006,1,0,E00.1,NaN,3100.0,NaN,0.0,1999.0,2022.0,3007,190.0,4.0,1.0,2024-06-05
7,1000007,1,0,E00.1,NaN,3100.0,NaN,1.0,1999.0,NaN,3004,200.0,5.0,1.0,2024-06-05
8,1000008,1,0,E00.1,NaN,3100.0,NaN,1.0,1999.0,NaN,3004,300.0,8.0,1.0,2024-06-05
9,1000009,1,0,E00.1,NaN,3100.0,NaN,1.0,1999.0,NaN,3004,196.0,7.0,1.0,2024-06-05


In [60]:
geneva_dwellings = metadata_translation(geneva_dwel,swapped_data_dict)
geneva_dwellings

,EGID,EWID,EDID,floor,multipleFloor,yearOfConstruction,dwellingStatus,surfaceAreaOfDwelling,noOfHabitableRooms,kitchen
0,203840,1,0,3100.0,1.0,NaN,3007,160.0,6.0,1.0
1,1000001,2,0,3100.0,1.0,1999.0,3004,160.0,6.0,1.0
2,1000002,1,0,3100.0,1.0,1999.0,3004,60.0,4.0,1.0
3,1000003,1,0,3100.0,0.0,1999.0,3007,96.0,4.0,1.0
4,1000004,1,0,3100.0,1.0,1999.0,3007,277.0,9.0,1.0
5,1000005,1,0,3100.0,0.0,1999.0,3007,90.0,3.0,1.0
6,1000006,1,0,3100.0,0.0,1999.0,3007,190.0,4.0,1.0
7,1000007,1,0,3100.0,1.0,1999.0,3004,200.0,5.0,1.0
8,1000008,1,0,3100.0,1.0,1999.0,3004,300.0,8.0,1.0
9,1000009,1,0,3100.0,1.0,1999.0,3004,196.0,7.0,1.0


In [61]:
add_nodes_to_graph(geneva_dwellings, 'dwelling')

C:\Users\User\AppData\Local\Temp\ipykernel_25024\3893174686.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node,label, **properties)


In [62]:
query = """
        MATCH (b:building), (d:dwelling)
        WHERE b.EGID = d.EGID
        MERGE (b)-[:HasDwelling]->(d)
    """

add_relationships_to_graph(query)

C:\Users\User\AppData\Local\Temp\ipykernel_25024\3939598042.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship,query)


<h2>Status Describes Building relationship</h2>
Another relationship that we can create, is the one that shows the status of the building.

In [63]:
status_dict = {
    'GSTAT': [1001, 1002, 1003, 1004, 1005, 1007, 1008],
    'statusDescriptionOfBuilding': [
        'En projet', 
        'Autorisé', 
        'En construction', 
        'Existant', 
        'Non utilisable', 
        'Démoli', 
        'Non réalisé'
    ]
}

swapped_data_dict['statusDescriptionOfBuilding'] = 'statusDescriptionOfBuilding'

In [64]:
status = pd.DataFrame(status_dict)
status

,GSTAT,statusDescriptionOfBuilding
0,1001,En projet
1,1002,Autorisé
2,1003,En construction
3,1004,Existant
4,1005,Non utilisable
5,1007,Démoli
6,1008,Non réalisé


In [65]:
bld_status = metadata_translation(status,swapped_data_dict)
bld_status

,buildingStatus,statusDescriptionOfBuilding
0,1001,En projet
1,1002,Autorisé
2,1003,En construction
3,1004,Existant
4,1005,Non utilisable
5,1007,Démoli
6,1008,Non réalisé


In [66]:
add_nodes_to_graph(bld_status, 'status')

C:\Users\User\AppData\Local\Temp\ipykernel_25024\3893174686.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node,label, **properties)


In [67]:
query = """
        MATCH (s:status), (b:building)
        WHERE s.buildingStatus = b.buildingStatus
        MERGE (s)-[:describesBuilding]->(b)
    """

add_relationships_to_graph(query)

C:\Users\User\AppData\Local\Temp\ipykernel_25024\3939598042.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship,query)


<h2>Canton contains Building relationship</h2>
Now we will study the relationship of the Building and the Canton. To be more precise, we will create a dataframe that will contain the canton abbrevation and which building they belong to, so the EGID.

In [68]:
cantons = {
    "AG": "Aargau",
    "AI": "Appenzell Innerrhoden",
    "AR": "Appenzell Ausserrhoden",
    "BE": "Bern",
    "BL": "Basel-Landschaft",
    "BS": "Basel-Stadt",
    "FR": "Fribourg",
    "GE": "Geneva",
    "GL": "Glarus",
    "GR": "Graubünden",
    "JU": "Jura",
    "LU": "Lucerne",
    "NE": "Neuchâtel",
    "NW": "Nidwalden",
    "OW": "Obwalden",
    "SG": "St. Gallen",
    "SH": "Schaffhausen",
    "SO": "Solothurn",
    "SZ": "Schwyz",
    "TG": "Thurgau",
    "TI": "Ticino",
    "UR": "Uri",
    "VD": "Vaud",
    "VS": "Valais",
    "ZG": "Zug",
    "ZH": "Zurich"
}

swapped_data_dict["cantonDescriptionFull"] = "cantonDescriptionFull"
canton = pd.DataFrame(list(cantons.items()), columns=['GDEKT', 'cantonDescriptionFull']) #convert to a dataframe
canton_df = metadata_translation(canton,swapped_data_dict)
canton_df

,cantonAbbreviation,cantonDescriptionFull
0,AG,Aargau
1,AI,Appenzell Innerrhoden
2,AR,Appenzell Ausserrhoden
3,BE,Bern
4,BL,Basel-Landschaft
5,BS,Basel-Stadt
6,FR,Fribourg
7,GE,Geneva
8,GL,Glarus
9,GR,Graubünden


In [69]:
add_nodes_to_graph(canton_df, 'canton')

C:\Users\User\AppData\Local\Temp\ipykernel_25024\3893174686.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node,label, **properties)


In [70]:
query = """
        MATCH  (c:canton),(b:building)
        WHERE c.cantonAbbreviation = b.cantonAbbreviation	
        MERGE (c)-[:ContainsBuilding]->(b)
    """

add_relationships_to_graph(query)

C:\Users\User\AppData\Local\Temp\ipykernel_25024\3939598042.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship,query)


<h2>Proving our concept with more data</h2>
All the above were done only with focus to the canton of Geneva. Now we will try to add more information to our database by conducting random sampling. We can not insert all the data since the storage that we have is not enough, as a result random sampling, is a good alternative in order to get variety of data.

In [71]:
all_buildings_df = pd.read_csv('C:\\Users\\User\\Desktop\\KOs\\ch\\gebaeude_batiment_edificio.csv',sep='\t')
all_buildings_df = all_buildings_df.sample(n=200)
all_buildings_df

C:\Users\User\AppData\Local\Temp\ipykernel_25024\281343177.py:1: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  all_buildings_df = pd.read_csv('C:\\Users\\User\\Desktop\\KOs\\ch\\gebaeude_batiment_edificio.csv',sep='\t')


,EGID,GDEKT,GGDENR,GGDENAME,EGRID,LGBKR,LPARZ,LPARZSX,LTYP,GEBNR,...,GWAERDATH2,GWAERZW1,GENW1,GWAERSCEW1,GWAERDATW1,GWAERZW2,GENW2,GWAERSCEW2,GWAERDATW2,GEXPDAT
2931107,190211786,VS,6082,Ayent,NaN,0.0,85,NaN,NaN,NaN,...,NaN,7650.0,7560.0,869.0,2009-11-10,NaN,NaN,NaN,NaN,2024-08-02
3188548,295073119,GE,6639,Soral,CH796581806329,42.0,10237.0,NaN,NaN,224,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-02
2774367,802837,VD,5671,Dompierre (VD),CH974580178365,0.0,26,NaN,NaN,29,...,2004-01-01,7630.0,7530.0,855.0,2004-01-01,7600.0,7500.0,855.0,2004-01-01,2024-08-02
135528,210298028,ZH,131,Adliswil,CH689206517766,0.0,4346,NaN,NaN,n.n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-02
439098,1233782,BE,351,Bern,CH980546358751,3.0,1314,NaN,NaN,NaN,...,NaN,7630.0,7530.0,860.0,2001-11-29,7600.0,7500.0,860.0,2001-11-29,2024-08-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305850,1535049,FR,2275,Murten,CH469110948751,2260.0,110,NaN,NaN,NaN,...,NaN,7630.0,7530.0,855.0,2014-12-31,NaN,NaN,NaN,NaN,2024-08-02
1917816,191996090,SG,3443,Gossau (SG),CH732177937341,0.0,696,NaN,NaN,1246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-02
816704,502048996,BE,941,Thierachern,CH334695359871,0.0,273,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-02
1717654,500003388,SG,3203,St. Gallen,CH975977098725,3207.0,F2670,NaN,NaN,f2174,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-02


Find the entrances and dwellings that are interesting for us depending on our random sampling.

In [72]:
entrances_df = pd.read_csv('C:\\Users\\User\\Desktop\\KOs\\ch\\eingang_entree_entrata.csv',sep='\t')

id_list = all_buildings_df['EGID'].tolist()

# Step 2: Construct the dynamic where clause
# Note: To prevent SQL injection, ensure that the ids are safe and clean.
query_str = "EGID in @id_list"

# Step 3: Apply the query to filter the DataFrame
entrances_df = entrances_df.query(query_str)
entrances_df

,EGID,EDID,EGAID,DEINR,ESID,STRNAME,STRNAMK,STRINDX,STRSP,STROFFIZIEL,DPLZ4,DPLZZ,DPLZNAME,DKODE,DKODN,DOFFADR,DEXPDAT
6264,6979,0,100014378,37,10003786.0,Bäumlisächerstrasse,Bäumlisächerstr.,Bäu,9901.0,1.0,8907,0,Wettswil,2677854.641,1244258.745,1.0,2024-08-02
26093,26633,0,100033973,3,10111253.0,Hofackerstrasse,Hofackerstr.,Hof,9901.0,1.0,8304,0,Wallisellen,2687658.547,1252815.880,1.0,2024-08-02
54442,55001,0,100054388,59,10003145.0,Kirchstrasse,Kirchstr.,Kir,9901.0,1.0,8810,0,Horgen,2687605.330,1234877.853,1.0,2024-08-02
67038,67765,0,100067903,7,10000862.0,Feldstrasse,Feldstr.,Fel,9901.0,1.0,8703,0,Erlenbach ZH,2688262.753,1239006.269,1.0,2024-08-02
77247,78044,0,100074122,31,10000945.0,Oberhausenstrasse,Oberhausenstr.,Obe,9901.0,1.0,8712,0,Stäfa,2696929.352,1233183.726,1.0,2024-08-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3260943,504123767,0,103172616,1a,10248195.0,Chemin de Chalosy,Ch. de Chalosy,Cha,9903.0,1.0,1772,2,Ponthaux,2569008.315,1184485.511,1.0,2024-08-02
3286016,504148970,0,103344703,2.1,10256708.0,Place de La Poste,Place de La Poste,Pce,9903.0,1.0,1984,0,Les Haudères,2605414.302,1103364.125,NaN,2024-08-02
3308200,504171313,0,103468442,269.1,10083340.0,Trémalmont,Trémalmont,Tré,9903.0,1.0,2124,0,Les Sagnettes,2536768.060,1199175.112,0.0,2024-08-02
3328286,504191717,0,103537483,75a,10081305.0,Route de Saxé,Rte de Saxé,Sax,9903.0,1.0,1926,0,Fully,2576357.595,1111018.778,NaN,2024-08-02


In [73]:
dwel = pd.read_csv('C:\\Users\\User\\Desktop\\KOs\\ch\\wohnung_logement_abitazione.csv',sep='\t')
dwel = dwel.query(query_str)
dwel

C:\Users\User\AppData\Local\Temp\ipykernel_25024\2790971887.py:1: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  dwel = pd.read_csv('C:\\Users\\User\\Desktop\\KOs\\ch\\wohnung_logement_abitazione.csv',sep='\t')


,EGID,EWID,EDID,WHGNR,WEINR,WSTWK,WBEZ,WMEHRG,WBAUJ,WABBJ,WSTAT,WAREA,WAZIM,WKCHE,WEXPDAT
11264,6979,1,0,1,NaN,3100.0,links,0.0,1999.0,NaN,3004.0,94.0,4.0,1.0,2024-08-02
11265,6979,2,0,2,NaN,3100.0,rechts,0.0,1999.0,NaN,3004.0,67.0,2.0,1.0,2024-08-02
11266,6979,3,0,101,NaN,3101.0,links,0.0,1999.0,NaN,3004.0,94.0,4.0,1.0,2024-08-02
11267,6979,4,0,102,NaN,3101.0,rechts,0.0,1999.0,NaN,3004.0,67.0,2.0,1.0,2024-08-02
11268,6979,5,0,201,NaN,3102.0,links,0.0,1999.0,NaN,3004.0,94.0,4.0,1.0,2024-08-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991026,280079017,1,1,NaN,NaN,3100.0,NaN,0.0,2012.0,NaN,3004.0,100.0,4.0,1.0,2024-08-02
4991027,280079017,2,1,NaN,NaN,3101.0,NaN,0.0,2012.0,NaN,3004.0,100.0,4.0,1.0,2024-08-02
5046480,280128942,1,0,NaN,NaN,3100.0,NaN,1.0,2022.0,NaN,3004.0,195.0,5.0,1.0,2024-08-02
5057255,295040250,1,0,E00.1,NaN,3100.0,NaN,0.0,2004.0,NaN,3004.0,149.0,5.0,1.0,2024-08-02


In order to show how we can scale this proposal, we will construct a dictionary that will hold the description of the relationships and the queries that apply to them. This way we can extract them and use our generic functions for each of them.

In [74]:
relationships = {
    "building has entrance": """
        MATCH (b:building), (e:entrance)
        WHERE b.EGID = e.EGID
        MERGE (b)-[:HasEntrance]->(e)
    """,
    "building has dwelling": """
        MATCH (b:building), (d:dwelling)
        WHERE b.EGID = d.EGID
        MERGE (b)-[:HasDwelling]->(d)
    """,
    "canton contains building": """
        MATCH  (c:canton), (b:building)
        WHERE c.cantonAbbreviation = b.cantonAbbreviation    
        MERGE (c)-[:ContainsBuilding]->(b)
    """,
    "status describes building": """
        MATCH (s:status), (b:building)
        WHERE s.buildingStatus = b.buildingStatus
        MERGE (s)-[:describesBuilding]->(b)
    """
}

entities = ["building","entrance","dwelling","status","canton"]

# Rename the dataframes
building = all_buildings_df
entrance = entrances_df
dwelling = dwel
status = bld_status
canton = canton_df


def get_dataframe_names(name):
    global_vars = globals()
    dataframe_names = [name for name, value in global_vars.items() if isinstance(value, pd.DataFrame)]
    if name in dataframe_names:
        return globals()[name]
    return 

for key in relationships:
    keys = key.split(" ")
    if (len(keys)) != 3:
        print("Error! You are not inserting a triplet!")
        break
    else:
        if keys[0] and keys[2] not in entities:
            print(keys[2])
            break
        else:
            df = metadata_translation(get_dataframe_names(keys[0]),swapped_data_dict)
            add_nodes_to_graph(df, keys[0])
            df1 = metadata_translation(get_dataframe_names(keys[2]),swapped_data_dict)
            add_nodes_to_graph(df1, keys[2])
            add_relationships_to_graph(relationships[key])
            print("Completed adding relationship:")
            print(" ".join(key))
            
            

C:\Users\User\AppData\Local\Temp\ipykernel_25024\3893174686.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node,label, **properties)
C:\Users\User\AppData\Local\Temp\ipykernel_25024\3893174686.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node,label, **properties)


Completed adding relationship:
b u i l d i n g   h a s   e n t r a n c e


C:\Users\User\AppData\Local\Temp\ipykernel_25024\3893174686.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node,label, **properties)
C:\Users\User\AppData\Local\Temp\ipykernel_25024\3893174686.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node,label, **properties)


Completed adding relationship:
b u i l d i n g   h a s   d w e l l i n g


C:\Users\User\AppData\Local\Temp\ipykernel_25024\3893174686.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node,label, **properties)
C:\Users\User\AppData\Local\Temp\ipykernel_25024\3893174686.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node,label, **properties)


Completed adding relationship:
c a n t o n   c o n t a i n s   b u i l d i n g


C:\Users\User\AppData\Local\Temp\ipykernel_25024\3893174686.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node,label, **properties)
C:\Users\User\AppData\Local\Temp\ipykernel_25024\3893174686.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node,label, **properties)


Completed adding relationship:
s t a t u s   d e s c r i b e s   b u i l d i n g
